This notebook converts the csv files to pickle files for quick processing in python.
Prerequisites: all the csv files of the mimics-iii database is dumped into a folder

In [ ]:
from glob import glob
import pandas as pd
import os

folder_containing_mimics_csv_files = os.environ.get("MIMICSPATH")


In [ ]:

filelist = sorted(glob(f"{folder_containing_mimics_csv_files}/*.csv"), key=os.path.getsize)
print(filelist)
for file_path in filelist:
    os.makedirs("../data",exist_ok=True)
    op_file = os.path.join("../data/", os.path.basename(file_path).replace(".csv", ".pkl"))
    print(f"Processing {file_path}...")
    if os.path.exists(op_file):
        print(f"Skipping {file_path}, already processed.")
        continue
    if "ADMISSIONS.csv" in file_path:
        parse_cols =  ["ADMITTIME", "DISCHTIME"]
    else:
        parse_cols = None
        
    chunks = pd.read_csv(file_path, parse_dates=parse_cols, chunksize=10000)
    all_chunks = []
    for chunk in chunks:
        all_chunks.append(chunk)
    combined_df = pd.concat(all_chunks)
    if "HADM_ID" in combined_df.columns:
        combined_df = combined_df[~combined_df.HADM_ID.isna()]
        combined_df["HADM_ID"] = combined_df["HADM_ID"].astype(int)
    combined_df.to_pickle(op_file)


In [ ]:
# Display all columns in all the files
import os
import pandas as pd

# Path to the folder containing pickle files
folder_path = "../data"

# Iterate through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".pkl"):  # Check if the file is a pickle file
        file_path = os.path.join(folder_path, file_name)
        try:
            # Load the pickle file into a DataFrame
            df = pd.read_pickle(file_path)
            print(f"Columns in {file_name}:")
            print(df.columns.tolist())
            print()
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
